# Laminar Newtonian Similarity

As usual, we will begin by importing the necessary python packages

In [ ]:
from barennet import SimilarityModel
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')

We will find the incomplete similarity exponents for the laminar newtonian 
mean velocity profile in wall coordinates. We know that the dimensionless quantities
involved are $(u^+, y^+, Re_\tau)$. They are related through the following equation:

$$ u^+ = y^+ - \frac{1}{2 Re_\tau} \left( y^+ \right)^2.$$

We provide a data file generated following the equation above with $10 < Re_\tau < 100$
and $0 < y^+ < Re_\tau$ (which, in bulk coordinates, translates to $0 < y < \delta$).
The MDDP construction of the three dimensionless quantities is also provided in the 
code cell below.

In [ ]:
# First we will prepare the inputs for the similarity model class for the 
# Bingham friction factor.

data_path = "../Data/laminar_MVP_data.xlsx"
dimensionally_independent_parameters = ["mu", "rho", "delta"]
dimensionally_dependent_parameters = ["y", "- dp/dz"]
dimensional_qoi = "u"
non_dimensional_parameters = ["y+", "Re_tau"]
non_dimensional_qoi = "u+"
non_dimensional_params_construction = { 
    "y+": {"y": 1.0, "- dp/dz": 0.5, "mu": -1, "rho": 0.5, "delta": 0.5},
    "Re_tau": {"y": 0.0, "- dp/dz": 0.5, "mu": -1, "rho": 0.5, "delta": 1.5}, 
}
non_dimensional_qoi_construction = {
    "u+": {"u": 1, "y": 0.0, "- dp/dz": -0.5, "mu": 0.0, "rho": 0.5, "delta": -0.5}
}
non_similar_params = ["y+"]
similar_params = ["Re_tau"]

In [ ]:
similarity_model = SimilarityModel(
    data_path=data_path,
    dimensionally_independent_params=dimensionally_independent_parameters,
    dimensionally_dependent_params=dimensionally_dependent_parameters,
    dimensional_qoi=dimensional_qoi,
    non_dimensional_params=non_dimensional_parameters,
    non_dimensional_qoi=non_dimensional_qoi,
    non_dimensional_params_construction=non_dimensional_params_construction,
    non_dimesional_qoi_construction=non_dimensional_qoi_construction,
    non_similar_params=non_similar_params,
    similar_params=similar_params
)

### Buckingham's Similarity Group

As soon as the dimensionless construction is provided and we create an instance 
of the SimilarityModel class, the Buckingham's similarity group can be calculated
by solving a linear system (chapter 5 of the thesis).
It shoud read:
\begin{align*}
    &\mu^* = A_1 \mu, \ \ \ \rho^* = A_2 \rho, \ \ \ \delta^* = A_3 \delta \\ & \\
    & \left( \frac{dp}{dz} \right)^* = \frac{A_1^2}{A_2 A_3^3} \left( \frac{dp}{dz} \right) \\ & \\
    &U^* = \frac{A_1}{A_2 A_3} U
\end{align*}

And, as you can see, this is exactly the output of the cell below.

In [ ]:
similarity_model.print_buckingham_similarity_group()

In [ ]:
similarity_model.find_incomplete_similarity(n_tries=5, n_epochs=1000, tol=5e-3)

In [ ]:
similarity_model.incomplete_similarity_exponents_dict

In [ ]:
similarity_model.print_renormalization_group()